# Parse_pdb_files

Parse pdb files and determine for each amino acid how often
it participates in different secondary structures. Write results
as tsv file.

PDB file format: https://www.wwpdb.org/documentation/file-format-content/format33/sect1.html

A PDB file consists of multiple sections: Title, Primary structure, Heterogen
section, Secondary structure, Connectivity annotation, etc.

Primary structure section
https://www.wwpdb.org/documentation/file-format-content/format33/sect3.html
Contains the sequence of residues in each chain of the macromolecule.

Secondary structure section
https://www.wwpdb.org/documentation/file-format-content/format33/sect5.html
Describes helices, sheets, and turns found in protein and polypeptide
structures. There are three possible records: HELIX, TURN and SHEET

One possibility is to write a parser for PDB files that extracts the
information from the "secondary structure section". Alternatively,
we can use tools like DSSP or other secondary structure prediction tools
to determine the sec. structure from atomic positions. Some possibly
relevant links:
- https://www.biostars.org/p/48/
- https://www.biostars.org/p/65055/
- https://www.researchgate.net/post/Is_there_a_way_to_search_the_PDB_for_secondary_structure_followed_by_a_sequence_motif
- https://en.wikipedia.org/wiki/Protein_secondary_structure
- https://en.wikipedia.org/wiki/DSSP_(hydrogen_bond_estimation_algorithm)

Biopython PDB package:
- https://biopython.org/docs/latest/api/Bio.PDB.html
- https://github.com/biopython/biopython/tree/master/Bio/PDB
- https://biopython.org/wiki/The_Biopython_Structural_Bioinformatics_FAQ
- http://biopython.org/DIST/docs/tutorial/Tutorial.html#sec182

The Biopython tutorial suggests to use DSSP to obtain secondary structure info, so
that seems like a reasonable choice.

In [1]:
import logging
import argparse
import collections
from pathlib import Path
from Bio.PDB import PDBParser, DSSP
from Bio import Data
import pandas as pd
import numpy as np

In [2]:
protein_letters = Data.CodonTable.IUPACData.protein_letters

# Solution 1
single files not implemented

In [21]:
def create_pdb_list():
    """Get list of pdb structure names."""
    pdb_list = []
    with open("downloaded_pdb_files.txt", "r") as f:
        for line in f:
            l = line.strip()
            l = l.lower()
            pdb_list.append(l)
    f.close()
    return pdb_list

In [5]:
# for testing
pdb_list = ['131l', '1a74']

#pdb_list = create_pdb_list()
len(pdb_list)

2

In [6]:
def parse_all_structure_files(pdb_list):
    """Parses relevant information of all structures and saves it in a tuple list."""
    # create dict for grouping in three main structures
    structure_dict = {'H':'Helix','G':'Helix','I':'Helix','B':'Sheet','E':'Sheet','T':'Other','S':'Other','-':'Other'}
    # create storage for tuples
    tuple_list = []
    # parse data from strucutre files
    p = PDBParser()    
    for pdb in pdb_list:
        structure = p.get_structure(pdb, f'.pdb{pdb}.pdb')
        # use only the first model
        model = structure[0]
        # calculate DSSP
        dssp = DSSP(model, f'./pdb{pdb}.pdb')
        # parse tuples from DSSP object
        for i in range (len(dssp)):
            a_key = dssp[dssp.keys()[i]]
            tup = (a_key[1], structure_dict[a_key[2]])
            tuple_list.append(tup)
    return tuple_list

In [7]:
tuple_list = parse_all_structure_files(pdb_list)
tuple_list[:10]

/apps/anaconda3/envs/py3/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 3824.
  warnings.warn(
/apps/anaconda3/envs/py3/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 3826.
  warnings.warn(
/apps/anaconda3/envs/py3/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 3828.
  warnings.warn(
/apps/anaconda3/envs/py3/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain D is discontinuous at line 3852.
  warnings.warn(
/apps/anaconda3/envs/py3/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 3875.
  warnings.warn(
/apps/anaconda3/envs/py3/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is d

[('M', 'Other'),
 ('N', 'Other'),
 ('I', 'Helix'),
 ('F', 'Helix'),
 ('E', 'Helix'),
 ('M', 'Helix'),
 ('L', 'Helix'),
 ('R', 'Helix'),
 ('I', 'Helix'),
 ('D', 'Helix')]

In [47]:
# Counter --> validation
collections.Counter(tuple_list)

Counter({('M', 'Other'): 1,
         ('N', 'Other'): 14,
         ('I', 'Helix'): 10,
         ('F', 'Helix'): 6,
         ('E', 'Helix'): 11,
         ('M', 'Helix'): 4,
         ('L', 'Helix'): 15,
         ('R', 'Helix'): 12,
         ('D', 'Helix'): 14,
         ('G', 'Other'): 35,
         ('L', 'Other'): 17,
         ('R', 'Sheet'): 7,
         ('L', 'Sheet'): 11,
         ('K', 'Sheet'): 8,
         ('I', 'Sheet'): 6,
         ('Y', 'Sheet'): 6,
         ('D', 'Other'): 4,
         ('T', 'Other'): 15,
         ('E', 'Other'): 3,
         ('Y', 'Other'): 5,
         ('S', 'Sheet'): 4,
         ('G', 'Sheet'): 9,
         ('I', 'Other'): 6,
         ('H', 'Sheet'): 11,
         ('T', 'Sheet'): 11,
         ('K', 'Other'): 5,
         ('S', 'Other'): 8,
         ('P', 'Other'): 21,
         ('N', 'Helix'): 13,
         ('A', 'Helix'): 19,
         ('K', 'Helix'): 8,
         ('S', 'Helix'): 5,
         ('R', 'Other'): 8,
         ('V', 'Sheet'): 7,
         ('Q', 'Helix'): 6,
     

In [20]:
def count_AA_structure_pairs(tuple_list):
    """Loops (AminoAcid, strucuture) tuples and counts it to matrix."""
    # create numpy array with zeros
    count_table = np.zeros(shape=(20,3))
    count_table
    # dicts to convert AA and structure to numpy coordinates
    AA_pos_dict = {'A': 0,'C': 1,'D': 2,'E': 3,'F': 4,'G': 5,'H': 6,'I': 7,'K': 8,'L': 9,'M': 10,'N': 11,'P': 12,'Q': 13,'R': 14,'S': 15,'T': 16,'V': 17,'W': 18,'Y': 19}
    struct_pos_dict = {'Helix': 0, 'Sheet': 1, 'Other': 2}
    # adds counting to matrix
    for i, j in tuple_list: # i is AA, j is structure
        count_table[AA_pos_dict[i], struct_pos_dict[j]] += 1
    return count_table

In [13]:
count_table = count_AA_structure_pairs(tuple_list)
count_table

array([[19.,  6., 11.],
       [ 0.,  7.,  9.],
       [14.,  0.,  4.],
       [11.,  4.,  3.],
       [ 6.,  2.,  5.],
       [ 5.,  9., 35.],
       [ 2., 11., 10.],
       [10.,  6.,  6.],
       [ 8.,  8.,  5.],
       [15., 11., 17.],
       [ 4.,  0.,  1.],
       [13.,  4., 14.],
       [ 6.,  2., 21.],
       [ 6.,  4., 11.],
       [12.,  7.,  8.],
       [ 5.,  4.,  8.],
       [ 7., 11., 15.],
       [14.,  7., 16.],
       [ 6.,  6.,  1.],
       [ 3.,  6.,  5.]])

In [31]:
def calc_count_table(count_table):
    """Takes a count_table as input and calculates percentage for each entry like (value/total_value)*100.
    Rounds result to 2 decimals."""
    # calculate sum of count_table
    sum_AA = np.sum(count_table)
    # calculate percentage from count
    freq_table = (count_table / sum_AA) * 100
    freq_table = np.round(freq_table, decimals=2)
    return freq_table

In [32]:
freq_table = calc_count_table(count_table)
# proof if calculation of freq_table is right
print(np.sum(freq_table))
freq_table

99.95000000000002


array([[3.91, 1.23, 2.26],
       [0.  , 1.44, 1.85],
       [2.88, 0.  , 0.82],
       [2.26, 0.82, 0.62],
       [1.23, 0.41, 1.03],
       [1.03, 1.85, 7.2 ],
       [0.41, 2.26, 2.06],
       [2.06, 1.23, 1.23],
       [1.65, 1.65, 1.03],
       [3.09, 2.26, 3.5 ],
       [0.82, 0.  , 0.21],
       [2.67, 0.82, 2.88],
       [1.23, 0.41, 4.32],
       [1.23, 0.82, 2.26],
       [2.47, 1.44, 1.65],
       [1.03, 0.82, 1.65],
       [1.44, 2.26, 3.09],
       [2.88, 1.44, 3.29],
       [1.23, 1.23, 0.21],
       [0.62, 1.23, 1.03]])

In [37]:
def add_AA_to_table(freq_table):
    """Creates AA numpy array and adds it to the input table."""
    # create column for AA's
    AA_list = np.array([['A'], ['C'], ['D'], ['E'], ['F'], ['G'], ['H'], ['I'], ['K'], ['L'], ['M'], ['N'], ['P'], ['Q'], ['R'], ['S'], ['T'], ['V'], ['W'], ['Y']])
    # append column to numpy array
    AA_freq_table = np.append(AA_list, freq_table, axis=1)
    return AA_freq_table

In [38]:
AA_freq_table = add_AA_to_table(freq_table)
AA_freq_table

array([['A', '3.91', '1.23', '2.26'],
       ['C', '0.0', '1.44', '1.85'],
       ['D', '2.88', '0.0', '0.82'],
       ['E', '2.26', '0.82', '0.62'],
       ['F', '1.23', '0.41', '1.03'],
       ['G', '1.03', '1.85', '7.2'],
       ['H', '0.41', '2.26', '2.06'],
       ['I', '2.06', '1.23', '1.23'],
       ['K', '1.65', '1.65', '1.03'],
       ['L', '3.09', '2.26', '3.5'],
       ['M', '0.82', '0.0', '0.21'],
       ['N', '2.67', '0.82', '2.88'],
       ['P', '1.23', '0.41', '4.32'],
       ['Q', '1.23', '0.82', '2.26'],
       ['R', '2.47', '1.44', '1.65'],
       ['S', '1.03', '0.82', '1.65'],
       ['T', '1.44', '2.26', '3.09'],
       ['V', '2.88', '1.44', '3.29'],
       ['W', '1.23', '1.23', '0.21'],
       ['Y', '0.62', '1.23', '1.03']], dtype='<U32')

In [41]:
def create_dataframe(AA_freq_table):
    """Takes numpy array with added AA's and converts it to pandasDataFrame with headers."""
    # create pandasDataframe from numpy array
    df_freq_table = pd.DataFrame(AA_freq_table, columns=['AA', 'Helix', 'Sheet', 'Other'])
    return df_freq_table

In [42]:
df_freq_table = create_dataframe(AA_freq_table)
df_freq_table

,AA,Helix,Sheet,Other
0,A,3.91,1.23,2.26
1,C,0.0,1.44,1.85
2,D,2.88,0.0,0.82
3,E,2.26,0.82,0.62
4,F,1.23,0.41,1.03
5,G,1.03,1.85,7.2
6,H,0.41,2.26,2.06
7,I,2.06,1.23,1.23
8,K,1.65,1.65,1.03
9,L,3.09,2.26,3.5


In [44]:
# into main function
df_freq_table.to_csv("./results/AA_freq_table.tsv", sep="\t", index=False)

In [68]:
#np.savetxt(pdb+".tsv", AA_freq_table, delimiter="\t")

In [ ]:
# Write as a CSV file with headers on first line
#with open('out.csv', 'w') as fp:
#    fp.write(','.join(ar.dtype.names) + '\n')
#    np.savetxt(fp, ar, '%s', ',')

# Solution 2
not finished

In [105]:
aux_dict = {'H':'Helix','G':'Helix','I':'Helix','B':'Sheet','E':'Sheet','T':'Other','S':'Other','-':'Other'}

# Einzige Struktur zum testen
pdb_list = ['131l', '1a74']

# create dictionary for counting
AA_dict = collections.defaultdict(int)

# hier einfach nur die beiden Listen erstellt nach denen wir groupen wollten um gleich innerhalb eines DF zu zählen
p = PDBParser()
for pdb in pdb_list:
    structure = p.get_structure(pdb, f'./pdb{pdb}.pdb')
    # use only the first model
    model = structure[0]
    # calculate DSSP
    dssp = DSSP(model, f'./pdb{pdb}.pdb')
    # extract sequence and secondary structure from the DSSP tuple
    for z in range(len(dssp)):
        a_key = list(dssp.keys())[z]
        elm = ( dssp[a_key][1],aux_dict[dssp[a_key][2]])
        AA_dict[elm] += 1


/apps/anaconda3/envs/py3/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 3824.
  warnings.warn(
/apps/anaconda3/envs/py3/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 3826.
  warnings.warn(
/apps/anaconda3/envs/py3/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 3828.
  warnings.warn(
/apps/anaconda3/envs/py3/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain D is discontinuous at line 3852.
  warnings.warn(
/apps/anaconda3/envs/py3/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 3875.
  warnings.warn(
/apps/anaconda3/envs/py3/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is d

In [106]:
# Erstellen Dataframe aus dem Dict und dann groupen und pivoten scheint nicht schlecht zu funktionieren
AA_data = pd.Series(AA_dict).rename_axis(['AA', 'Structures']).reset_index(name='Counts')
sum_AA = AA_data["Counts"].sum()
AA_data = AA_data.pivot_table(values='Counts', index='AA', columns='Structures', aggfunc='first')
AA_data

Structures,Helix,Other,Sheet
AA,,,
A,19.0,11.0,6.0
C,NaN,9.0,7.0
D,14.0,4.0,NaN
E,11.0,3.0,4.0
F,6.0,5.0,2.0
G,5.0,35.0,9.0
H,2.0,10.0,11.0
I,10.0,6.0,6.0
K,8.0,5.0,8.0


In [107]:
#print(sum_AA)
AA_data = AA_data.fillna(0)
test = ((AA_data/sum_AA)*100).round(2)
test
test.to_csv("./results/test.tsv", sep="\t")